Clustering : K-Means

Importing the Essential Libraries and loading data :

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

df = pd.read_csv("ruspini.csv", sep = ",")
df.head(5)

,,,",""X"",""x"",""y"""
1,1,4,53
2,2,5,63
3,3,10,59
4,4,9,77
5,5,13,49
